In [1]:
# 📌 Celda 1: Imports
from langchain.chat_models import init_chat_model
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.graph import START, MessagesState, StateGraph
from langgraph.prebuilt import ToolNode, tools_condition


In [2]:
# 📌 Celda 2: Definición de la clase procesadora
class NotebookMCPClient:
    def __init__(self):
        self.server_configs = {
            "assistance": {"url": "http://localhost:9001/sse", "transport": "sse"},
            "contracting": {"url": "http://localhost:9002/sse", "transport": "sse"},
            "voting": {"url": "http://localhost:9003/sse", "transport": "sse"},
        }
        self.client = None
        self.graph = None
        self.model = init_chat_model("openai:gpt-4.1")

    async def start(self):
        self.client = MultiServerMCPClient(self.server_configs)
        await self.client.__aenter__()

        tools = self.client.get_tools()
        print(f"🔧 Herramientas cargadas: {[t.name for t in tools]}")

        def call_model(state: MessagesState):
            response = self.model.bind_tools(tools).invoke(state["messages"])
            return {"messages": response}

        builder = StateGraph(MessagesState)
        builder.add_node("call_model", call_model)
        builder.add_node("tools", ToolNode(tools))
        builder.add_edge(START, "call_model")
        builder.add_conditional_edges("call_model", tools_condition)
        builder.add_edge("tools", "call_model")
        self.graph = builder.compile()

    async def stop(self):
        if self.client:
            await self.client.__aexit__(None, None, None)

    async def run(self, pregunta: str):
        if not self.graph:
            raise RuntimeError("Primero debes ejecutar `await client.start()`")
        return await self.graph.ainvoke({"messages": pregunta})


In [3]:
# 📌 Celda 3: Crear la instancia y levantarla
client = NotebookMCPClient()
await client.start()

🔧 Herramientas cargadas: ['obtener_asistencias_congresista', 'obtener_ranking_asistencia', 'listar_faltas_recurrentes', 'obtener_historial_asistencia', 'obtener_estadisticas_generales_asistencia', 'buscar_contratos_por_ruc', 'listar_contratos_recientes', 'obtener_entidades_top_por_proveedor', 'consultar_estado_proveedor', 'clasificar_contratos_por_categoria', 'buscar_votaciones_por_tema', 'resumen_votaciones_congresista', 'obtener_votaciones_agrupadas_por_tema', 'obtener_participacion_votaciones', 'identificar_votos_clave_por_tema']


In [4]:
# 📌 Celda 4: Función de utilidad para preguntar
async def preguntar_a_mcp(pregunta: str):
    respuesta = await client.run(pregunta)
    print("🧠 Respuesta:")
    for mensaje in respuesta["messages"]:
        print("-", mensaje.content)


In [5]:
# 📌 Celda 5: Ejemplo de uso
await preguntar_a_mcp("¿Cuántos contratos tiene la empresa con RUC 123456789?")

🧠 Respuesta:
- ¿Cuántos contratos tiene la empresa con RUC 123456789?
- 
- {
  "ruc": "123456789",
  "total_contratos": 42,
  "monto_total": 1250000.5,
  "entidades_top": [
    "MINSA",
    "Gobierno Regional de Lima",
    "MTC"
  ]
}
- La empresa con RUC 123456789 tiene un total de 42 contratos con el Estado.


In [7]:
await preguntar_a_mcp("Que herramientas tienes disponibles?")

🧠 Respuesta:
- Que herramientas tienes disponibles?
- Actualmente, tengo acceso a varias herramientas que me permiten obtener información sobre la actividad parlamentaria y la contratación con el Estado peruano. Algunas de las principales funcionalidades que puedo ofrecer son:

1. Asistencias y Faltas en el Congreso:
- Consultar el resumen de asistencias, faltas y licencias de cada congresista.
- Ranking de congresistas con mejor asistencia.
- Listar a congresistas con altas tasas de faltas.
- Estadísticas generales sobre la asistencia en el Congreso.
- Historial detallado de la asistencia de congresistas por legislaturas.

2. Contratos con el Estado Peruano:
- Buscar y listar contratos de proveedores del Estado usando su RUC.
- Ver los contratos más recientes de un proveedor.
- Consultar si un proveedor está habilitado para contratar con el Estado.
- Analizar los contratos por categoría (bienes, servicios, obras).
- Ver las entidades estatales que más han contratado con un proveedor.


In [9]:
# 📌 Celda 6: Cierre (opcional al terminar) --No comaptible con Jupyter
# await client.stop()¿